In [1]:
#Andhra pradesh

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "button")))
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']")))
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('ap_bus_details.csv', index=False)



In [6]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 4045,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",15:45,06h 15m,22:00,4.2,INR 436,28 Seats available
1,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3356,NIGHT RIDER (SEATER CUM SLEEPER),15:45,07h 00m,22:45,2.8,INR 639,32 Seats available
2,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3546,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),16:15,06h 45m,23:00,3.7,INR 670,42 Seats available
3,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3500,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",16:20,08h 25m,00:45,3.9,INR 469,20 Seats available
4,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3376,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",16:45,06h 30m,23:15,4.3,INR 436,32 Seats available
...,...,...,...,...,...,...,...,...,...,...
2614,Hyderabad to Kanigiri,https://www.redbus.in/bus-tickets/hyderabad-to...,Harikrishna Travels,Non A/C Seater / Sleeper (2+1),23:00,08h 00m,07:00,0,760,18 Seats available
2615,Hyderabad to Kanigiri,https://www.redbus.in/bus-tickets/hyderabad-to...,Lakshmi Travels,Non A/C Seater / Sleeper (2+1),19:30,10h 20m,05:50,0,2775,40 Seats available
2616,Hyderabad to Kanigiri,https://www.redbus.in/bus-tickets/hyderabad-to...,MVN Tours and Travels,Non A/C Seater / Sleeper (2+1),22:50,07h 00m,05:50,0,3800,36 Seats available
2617,Hyderabad to Kanigiri,https://www.redbus.in/bus-tickets/hyderabad-to...,Anish Travels,NON A/C Sleeper (2+1),22:40,07h 30m,06:10,0,1045,13 Seats available


In [7]:
#kerala

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "button")))
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 3):  # There are 2 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('kerala_bus_details.csv', index=False)



In [9]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to...,KSRTC (Kerala) - 2810,Super Fast Non AC Seater (2+3),15:25,05h 40m,21:05,3.1,INR 269,49 Seats available
1,Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to...,KSRTC (Kerala) - 720,Super Fast Non AC Seater (2+3),15:40,05h 35m,21:15,2.5,INR 269,50 Seats available
2,Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to...,KSRTC (Kerala) - 2898,Low Floor AC Seater 2+2,16:00,06h 15m,22:15,4.3,INR 383,7 Seats available
3,Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to...,KSRTC (Kerala) - 2874,Super Fast Non AC Seater (2+3),16:20,06h 10m,22:30,2.6,INR 230,49 Seats available
4,Kozhikode to Ernakulam,https://www.redbus.in/bus-tickets/kozhikode-to...,KSRTC (Kerala) - 3911,Super Fast Non AC Seater (2+3),17:25,05h 15m,22:40,3.2,INR 269,39 Seats available
...,...,...,...,...,...,...,...,...,...,...
684,Kalpetta (kerala) to Kozhikode,https://www.redbus.in/bus-tickets/kalpetta-to-...,Kottamala Tour And Travels,AC Seater (2+2),18:46,02h 00m,20:46,4.5,INR 3809.52,40 Seats available
685,Kalpetta (kerala) to Kozhikode,https://www.redbus.in/bus-tickets/kalpetta-to-...,Cochin Bus,Volvo Multi-Axle B9R A/c Semi Sleeper (2+2),22:25,01h 33m,23:58,4.2,INR 550,17 Seats available
686,Kalpetta (kerala) to Kozhikode,https://www.redbus.in/bus-tickets/kalpetta-to-...,Sonia Travels,A/C Seater/Sleeper (2+1),21:25,02h 04m,23:29,3.3,INR 450,18 Seats available
687,Kalpetta (kerala) to Kozhikode,https://www.redbus.in/bus-tickets/kalpetta-to-...,A1 Travels,Bharat Benz A/C Sleeper (2+1),21:00,02h 30m,23:30,4.5,540,19 Seats available


In [10]:
#telangana

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "button")))
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 4):  # There are 3 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('telangana_bus_details.csv', index=False)



In [12]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 4045,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",15:45,06h 15m,22:00,4.2,INR 436,31 Seats available
1,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3546,AMARAVATHI (VOLVO / SCANIA A.C Multi Axle),16:15,06h 45m,23:00,3.7,INR 670,42 Seats available
2,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3500,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",16:20,08h 25m,00:45,3.9,INR 469,16 Seats available
3,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 3376,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",16:45,06h 30m,23:15,4.3,INR 436,29 Seats available
4,Hyderabad to Vijayawada,https://www.redbus.in/bus-tickets/hyderabad-to...,APSRTC - 2561,INDRA(A.C. Seater),17:15,05h 15m,22:30,4.2,INR 528,33 Seats available
...,...,...,...,...,...,...,...,...,...,...
1142,Kodad to Hyderabad,https://www.redbus.in/bus-tickets/kodad-to-hyd...,TGSRTC - 7770,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",22:40,04h 10m,02:50,0,INR 289,29 Seats available
1143,Kodad to Hyderabad,https://www.redbus.in/bus-tickets/kodad-to-hyd...,TGSRTC - 42762,"SUPER LUXURY (NON-AC, 2 + 2 PUSH BACK)",23:20,02h 40m,02:00,0,INR 289,32 Seats available
1144,Kodad to Hyderabad,https://www.redbus.in/bus-tickets/kodad-to-hyd...,TGSRTC - 14562,eGARUDA,23:45,02h 45m,02:30,0,INR 351,35 Seats available
1145,Kodad to Hyderabad,https://www.redbus.in/bus-tickets/kodad-to-hyd...,NueGo,Electric A/C Seater (2+2),16:45,03h 50m,20:35,0,338,12 Seats available


In [4]:
# PUNJAB

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/pepsu-punjab"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "button")))
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 3):  # There are 2 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('Punjab_bus_details.csv', index=False)



In [6]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Delhi to Patiala,https://www.redbus.in/bus-tickets/delhi-to-pat...,PEPSU (Punjab) - 66665,Volvo AC Seater (2+2),14:00,05h 00m,19:00,4.0,INR 702,33 Seats available
1,Delhi to Patiala,https://www.redbus.in/bus-tickets/delhi-to-pat...,PEPSU (Punjab) - 66689,Volvo AC Seater (2+2),15:26,04h 59m,20:25,3.5,INR 702,36 Seats available
2,Delhi to Patiala,https://www.redbus.in/bus-tickets/delhi-to-pat...,PEPSU (Punjab) - 66667,Volvo AC Seater (2+2),17:31,04h 59m,22:30,4.1,INR 702,33 Seats available
3,Delhi to Patiala,https://www.redbus.in/bus-tickets/delhi-to-pat...,PEPSU (Punjab) - 67103,AC Seater Hvac 2+2,22:31,05h 29m,04:00,3.4,INR 445,31 Seats available
4,Delhi to Patiala,https://www.redbus.in/bus-tickets/delhi-to-pat...,Indo Canadian Transport Company,Mercedes A/C Semi Sleeper (2+2),16:30,07h 00m,23:30,4.9,INR 3300,46 Seats available
...,...,...,...,...,...,...,...,...,...,...
667,Delhi Airport to Phagwara,https://www.redbus.in/bus-tickets/delhi-airpor...,ADTC tour and Travels,Bharat Benz A/C Seater /Sleeper (2+1),22:40,06h 05m,04:45,3.9,INR 549,18 Seats available
668,Delhi Airport to Phagwara,https://www.redbus.in/bus-tickets/delhi-airpor...,ADTC tour and Travels,Volvo Multi-Axle I-Shift B11R Semi Sleeper (2+2),22:25,06h 15m,04:40,3.5,INR 699,8 Seats available
669,Delhi Airport to Phagwara,https://www.redbus.in/bus-tickets/delhi-airpor...,Banjara Satguru Travels,A/C Seater / Sleeper (2+1),22:40,07h 06m,05:46,3.5,INR 751,38 Seats available
670,Delhi Airport to Phagwara,https://www.redbus.in/bus-tickets/delhi-airpor...,Ram Dalal Holidays,Volvo 9600 Multi-Axle A/C Sleeper (2+1),21:30,07h 25m,04:55,0,INR 1790,20 Seats available


In [16]:
# himachal 

In [17]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "button")))
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 5):  # There are 4 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('himachal_bus_details.csv', index=False)



In [18]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 525,Ordinary 3+2 Non AC Seater,16:50,10h 10m,03:00,4.8,INR 512,27 Seats available
1,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 532,Ordinary 3+2 Non AC Seater,17:59,10h 16m,04:15,3.7,INR 512,27 Seats available
2,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 2040094,Ordinary 3+2 Non AC Seater,18:31,09h 44m,04:15,4.2,INR 512,36 Seats available
3,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 93,Himsuta AC Seater Volvo/Scania 2+2,19:35,09h 15m,04:50,4.3,INR 912,18 Seats available
4,Delhi to Shimla,https://www.redbus.in/bus-tickets/delhi-to-shimla,HRTC - 301,Ordinary 3+2 Non AC Seater,19:43,09h 47m,05:30,4.6,INR 512,27 Seats available
...,...,...,...,...,...,...,...,...,...,...
812,Delhi to Baddi (Himachal Pradesh),https://www.redbus.in/bus-tickets/delhi-to-bad...,HRTC - 568,Ordinary 3+2 Non AC Seater,21:30,06h 30m,04:00,4.0,INR 325,38 Seats available
813,Delhi to Baddi (Himachal Pradesh),https://www.redbus.in/bus-tickets/delhi-to-bad...,HRTC - 580,Ordinary 3+2 Non AC Seater,22:45,06h 15m,05:00,4.4,INR 325,37 Seats available
814,Delhi to Baddi (Himachal Pradesh),https://www.redbus.in/bus-tickets/delhi-to-bad...,HRTC - 576,A/C Executive (2+3),23:20,04h 20m,03:40,3.8,INR 430,35 Seats available
815,Delhi to Baddi (Himachal Pradesh),https://www.redbus.in/bus-tickets/delhi-to-bad...,TANISHQ HOLIDAYS TOUR,Volvo Multi-Axle A/C Semi Sleeper (2+2),19:00,06h 30m,01:30,3.2,INR 1799,49 Seats available


In [19]:
#uttar pradesh 

In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(8)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "button")))
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(8)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('up_bus_details.csv', index=False)



In [21]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Bareilly (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - RKD0119,Shatabdi AC Seater 2+2,16:16,05h 00m,21:16,3.3,INR 510,35 Seats available
1,Bareilly (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - KSB0086,Volvo AC Seater 2+2,17:01,05h 30m,22:31,4.6,INR 699,38 Seats available
2,Bareilly (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - RKD0130,Janrath AC Seater 2+3,17:31,05h 00m,22:31,2.3,INR 472,48 Seats available
3,Bareilly (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...,UPSRTC - BLY0111,Janrath AC Seater 2+2,19:01,04h 44m,23:45,3.2,INR 510,36 Seats available
4,Bareilly (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/bareilly-to-...,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...
1197,Delhi to Lucknow,https://www.redbus.in/bus-tickets/delhi-to-luc...,Laxmi holidays,Bharat Benz A/C Seater /Sleeper (2+1),20:30,08h 00m,04:30,4.4,INR 599,15 Seats available
1198,Delhi to Lucknow,https://www.redbus.in/bus-tickets/delhi-to-luc...,Vaishali Expresso Pvt. Ltd.,A/C Seater / Sleeper (2+1),18:00,08h 20m,02:20,4.4,INR 400,14 Seats available
1199,Delhi to Lucknow,https://www.redbus.in/bus-tickets/delhi-to-luc...,Fouji travels,A/C Seater / Sleeper (2+1),23:00,08h 00m,07:00,4.1,INR 300,25 Seats available
1200,Delhi to Lucknow,https://www.redbus.in/bus-tickets/delhi-to-luc...,Fouji travels,A/C Seater / Sleeper (2+1),20:30,08h 00m,04:30,4.3,INR 499,16 Seats available


In [22]:
# chandigarh

In [23]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/chandigarh-transport-undertaking-ctu"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "button")))
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 4):  # There are 3 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('chandigarh_bus_details.csv', index=False)



Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/dinanagar-punjab-to-chandigarh: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF685294C25+3179557]
	(No symbol) [0x00007FF684EF88A0]
	(No symbol) [0x00007FF684D891CA]
	(No symbol) [0x00007FF684DDFA67]
	(No symbol) [0x00007FF684DDFC9C]
	(No symbol) [0x00007FF684E33627]
	(No symbol) [0x00007FF684E07C6F]
	(No symbol) [0x00007FF684E302F3]
	(No symbol) [0x00007FF684E07A03]
	(No symbol) [0x00007FF684DD06D0]
	(No symbol) [0x00007FF684DD1983]
	GetHandleVerifier [0x00007FF6852F67CD+3579853]
	GetHandleVerifier [0x00007FF68530D1D2+3672530]
	GetHandleVerifier [0x00007FF685302153+3627347]
	GetHandleVerifier [0x00007FF68506092A+868650]
	(No symbol) [0x00007FF684F02FFF]
	(No symbol) [0x00007FF684EFF4A4]
	(No symbol) [0x00007FF684EFF646]
	(No symbol) [0x00007FF684EEEAA9]
	BaseThreadInitThunk [0x00007FF897BBE8D7+23]
	RtlUserThreadStart [0x00007FF899E1BF6C+44]

Error occurred while scraping bus details for https://www.red

In [24]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh Transport Undertaking (CTU) - 165720,HVAC Seater (2+3),15:30,05h 35m,21:05,4.6,INR 414.3,45 Seats available
1,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh Transport Undertaking (CTU) - 165722,HVAC Seater (2+3),16:30,05h 35m,22:05,3.0,INR 414.3,47 Seats available
2,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh Transport Undertaking (CTU) - 165724,HVAC Seater (2+3),17:00,05h 35m,22:35,3.1,INR 414.3,47 Seats available
3,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...,Chandigarh Transport Undertaking (CTU) - 165726,HVAC Seater (2+3),17:30,05h 35m,23:05,3.7,INR 414.3,47 Seats available
4,Chandigarh to Delhi,https://www.redbus.in/bus-tickets/chandigarh-t...,RSRTC - 192348,Super Luxury Volvo AC Seater Pushback 2+2,21:15,05h 15m,02:30,2.9,INR 691,30 Seats available
...,...,...,...,...,...,...,...,...,...,...
355,Chandigarh to Agra,https://www.redbus.in/bus-tickets/chandigarh-t...,Laxmi holidays,Bharat Benz A/C Seater /Sleeper (2+1),16:00,09h 20m,01:20,4.8,INR 848,5 Seats available
356,Chandigarh to Agra,https://www.redbus.in/bus-tickets/chandigarh-t...,GK Travels,A/C Seater/Sleeper (2+1),16:00,11h 00m,03:00,3.7,949,7 Seats available
357,Chandigarh to Agra,https://www.redbus.in/bus-tickets/chandigarh-t...,Vijay Tour and Travels,Bharat Benz A/C Seater /Sleeper (2+1),17:30,11h 00m,04:30,4.1,989,24 Seats available
358,Chandigarh to Agra,https://www.redbus.in/bus-tickets/chandigarh-t...,Gajraj bus service,Bharat Benz A/C Seater /Sleeper (2+1),15:50,07h 55m,23:45,2.2,629,13 Seats available


In [1]:
#kadamba

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/ktcl"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "button")))
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 4):  # There are 3 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('Kadamba_bus_details.csv', index=False)



Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/marcel-to-belagavi: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6B8BB4C25+3179557]
	(No symbol) [0x00007FF6B88188A0]
	(No symbol) [0x00007FF6B86A91CA]
	(No symbol) [0x00007FF6B86FFA67]
	(No symbol) [0x00007FF6B86FFC9C]
	(No symbol) [0x00007FF6B8753627]
	(No symbol) [0x00007FF6B8727C6F]
	(No symbol) [0x00007FF6B87502F3]
	(No symbol) [0x00007FF6B8727A03]
	(No symbol) [0x00007FF6B86F06D0]
	(No symbol) [0x00007FF6B86F1983]
	GetHandleVerifier [0x00007FF6B8C167CD+3579853]
	GetHandleVerifier [0x00007FF6B8C2D1D2+3672530]
	GetHandleVerifier [0x00007FF6B8C22153+3627347]
	GetHandleVerifier [0x00007FF6B898092A+868650]
	(No symbol) [0x00007FF6B8822FFF]
	(No symbol) [0x00007FF6B881F4A4]
	(No symbol) [0x00007FF6B881F646]
	(No symbol) [0x00007FF6B880EAA9]
	BaseThreadInitThunk [0x00007FF897BBE8D7+23]
	RtlUserThreadStart [0x00007FF899E1BF6C+44]

Error occurred while scraping bus details for https://www.redbus.in/bus-t

In [3]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Kadamba Transport Corporation Limited (KTCL) -...,A/C Sleeper (2+1),19:15,10h 35m,05:50,3.8,INR 1100,8 Seats available
1,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Kadamba Transport Corporation Limited (KTCL) -...,A/C Sleeper (2+1),20:30,09h 30m,06:00,4.5,INR 1100,5 Seats available
2,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Atmaram Gobus,VE A/C Sleeper (2+1),21:00,10h 30m,07:30,4.8,INR 1299,3 Seats available
3,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Laxmi holidays,Bharat Benz A/C Seater /Sleeper (2+1),21:25,10h 05m,07:30,4.7,INR 790,19 Seats available
4,Pune to Goa,https://www.redbus.in/bus-tickets/pune-to-goa,Atmaram Gobus,VE A/C Sleeper (2+1),22:30,10h 15m,08:45,4.6,INR 950,4 Seats available
...,...,...,...,...,...,...,...,...,...,...
395,Goa to Miraj,https://www.redbus.in/bus-tickets/goa-to-miraj,Khurana Travel Services,A/C Sleeper (2+1),15:15,08h 20m,23:35,3.8,INR 1100,23 Seats available
396,Goa to Miraj,https://www.redbus.in/bus-tickets/goa-to-miraj,Vaibhav Travels,A/C Sleeper (2+1),11:10,07h 50m,19:00,3.4,INR 399,9 Seats available
397,Goa to Miraj,https://www.redbus.in/bus-tickets/goa-to-miraj,KOKAN TOURS & TRAVELS,A/C Sleeper (2+1),20:30,09h 45m,06:15,1.8,INR 1000,30 Seats available
398,Goa Airport to Panaji,https://www.redbus.in/bus-tickets/goa-airport-...,Kadamba Transport Corporation Limited (KTCL) -...,AC Seater (2+2),13:45,01h 01m,14:46,3.9,INR 150,41 Seats available


In [ ]:
# north bengal

In [28]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/travels/nbstc"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "button")))
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('Northbengal_bus_details.csv', index=False)



Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/siliguri-to-darjeeling: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF685294C25+3179557]
	(No symbol) [0x00007FF684EF88A0]
	(No symbol) [0x00007FF684D891CA]
	(No symbol) [0x00007FF684DDFA67]
	(No symbol) [0x00007FF684DDFC9C]
	(No symbol) [0x00007FF684E33627]
	(No symbol) [0x00007FF684E07C6F]
	(No symbol) [0x00007FF684E302F3]
	(No symbol) [0x00007FF684E07A03]
	(No symbol) [0x00007FF684DD06D0]
	(No symbol) [0x00007FF684DD1983]
	GetHandleVerifier [0x00007FF6852F67CD+3579853]
	GetHandleVerifier [0x00007FF68530D1D2+3672530]
	GetHandleVerifier [0x00007FF685302153+3627347]
	GetHandleVerifier [0x00007FF68506092A+868650]
	(No symbol) [0x00007FF684F02FFF]
	(No symbol) [0x00007FF684EFF4A4]
	(No symbol) [0x00007FF684EFF646]
	(No symbol) [0x00007FF684EEEAA9]
	BaseThreadInitThunk [0x00007FF897BBE8D7+23]
	RtlUserThreadStart [0x00007FF899E1BF6C+44]

Error occurred while scraping bus details for https://www.redbus.in/b

In [29]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Kolkata to Siliguri,https://www.redbus.in/bus-tickets/kolkata-to-s...,Easy Ride,Volvo 9600 Multi-Axle A/C Sleeper (2+1),20:30,12h 55m,09:25,4.7,INR 2500,11 Seats available
1,Kolkata to Siliguri,https://www.redbus.in/bus-tickets/kolkata-to-s...,Shyamoli Paribahan Pvt Ltd,Volvo 9600 Multi-Axle A/C Sleeper (2+1),19:15,13h 00m,08:15,4.6,INR 2500,5 Seats available
2,Kolkata to Siliguri,https://www.redbus.in/bus-tickets/kolkata-to-s...,Easy Ride,Volvo 9600 Multi-Axle A/C Sleeper (2+1),19:50,12h 55m,08:45,4.7,INR 2500,1 Seat available
3,Kolkata to Siliguri,https://www.redbus.in/bus-tickets/kolkata-to-s...,Greenline,Volvo 9600 Multi-Axle A/C Sleeper (2+1),18:30,13h 00m,07:30,4.4,INR 2270,3 Seats available
4,Kolkata to Siliguri,https://www.redbus.in/bus-tickets/kolkata-to-s...,Express Line,Volvo 9600 Multi-Axle A/C Sleeper (2+1),18:30,13h 10m,07:40,4.4,INR 2399,2 Seats available
...,...,...,...,...,...,...,...,...,...,...
292,Balurghat to Siliguri,https://www.redbus.in/bus-tickets/balurghat-to...,Padmanabha Paribahan,A/C Seater / Sleeper (2+2),20:45,08h 25m,05:10,3.5,INR 350,13 Seats available
293,Balurghat to Siliguri,https://www.redbus.in/bus-tickets/balurghat-to...,Shree Krishna Travels,NON A/C Seater / Sleeper (2+2),19:45,09h 15m,05:00,3.1,INR 300,22 Seats available
294,Balurghat to Siliguri,https://www.redbus.in/bus-tickets/balurghat-to...,Mohanta Travels,NON A/C Seater / Sleeper (2+2),20:00,09h 00m,05:00,2.1,INR 300,43 Seats available
295,Raiganj to Ranaghat,https://www.redbus.in/bus-tickets/raiganj-to-r...,NBSTC-RAI-KOL-R1-19:30 - 156414,NON A/C Seater Push Back (2+2),20:30,08h 45m,05:15,4.3,INR 278,10 Seats available


In [ ]:
# bihar 

In [30]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/bihar-state-road-transport-corporation-bsrtc/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, "button")))
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 6):  # There are 5 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('bihar_bus_details.csv', index=False)



Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/delhi-to-motihari: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF685294C25+3179557]
	(No symbol) [0x00007FF684EF88A0]
	(No symbol) [0x00007FF684D891CA]
	(No symbol) [0x00007FF684DDFA67]
	(No symbol) [0x00007FF684DDFC9C]
	(No symbol) [0x00007FF684E33627]
	(No symbol) [0x00007FF684E07C6F]
	(No symbol) [0x00007FF684E302F3]
	(No symbol) [0x00007FF684E07A03]
	(No symbol) [0x00007FF684DD06D0]
	(No symbol) [0x00007FF684DD1983]
	GetHandleVerifier [0x00007FF6852F67CD+3579853]
	GetHandleVerifier [0x00007FF68530D1D2+3672530]
	GetHandleVerifier [0x00007FF685302153+3627347]
	GetHandleVerifier [0x00007FF68506092A+868650]
	(No symbol) [0x00007FF684F02FFF]
	(No symbol) [0x00007FF684EFF4A4]
	(No symbol) [0x00007FF684EFF646]
	(No symbol) [0x00007FF684EEEAA9]
	BaseThreadInitThunk [0x00007FF897BBE8D7+23]
	RtlUserThreadStart [0x00007FF899E1BF6C+44]

Error occurred while scraping bus details for https://www.redbus.in/bus-ti

In [31]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...,V.I.P Travels,A/C Seater / Sleeper (2+2),21:40,06h 00m,03:40,3.6,INR 350,33 Seats available
1,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...,Jai Hanuman Travels,A/C Seater / Sleeper (2+2),22:20,06h 40m,05:00,3.2,495,54 Seats available
2,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...,Om Sai Ram Bus,NON A/C Seater (2+2),17:55,04h 25m,22:20,2.9,INR 350,41 Seats available
3,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...,Om Sai Ram Bus,A/C Seater (2+2),18:10,04h 20m,22:30,2.9,INR 400,38 Seats available
4,Patna (Bihar) to Bettiah,https://www.redbus.in/bus-tickets/patna-to-bet...,Om Sai Ram Bus,NON A/C Seater (2+2),18:30,04h 30m,23:00,0,INR 350,41 Seats available
...,...,...,...,...,...,...,...,...,...,...
386,Ramgarh (Jharkhand) to Patna (Bihar),https://www.redbus.in/bus-tickets/ramgarh-jhar...,Tejas Maa Bhawani,A/C Sleeper (2+1),20:31,07h 19m,03:50,2.3,INR 800,26 Seats available
387,Ramgarh (Jharkhand) to Patna (Bihar),https://www.redbus.in/bus-tickets/ramgarh-jhar...,Tejas Maa Bhawani,Non AC Seater (2+2),19:45,08h 16m,04:01,2.3,INR 649,2 Seats available
388,Ramgarh (Jharkhand) to Patna (Bihar),https://www.redbus.in/bus-tickets/ramgarh-jhar...,Shakti Travels,NON A/C Seater / Sleeper (2+2),19:15,06h 45m,02:00,1.7,INR 600,59 Seats available
389,Ramgarh (Jharkhand) to Patna (Bihar),https://www.redbus.in/bus-tickets/ramgarh-jhar...,BD Parivahan,A/C Seater / Sleeper (2+2),22:40,07h 20m,06:00,1.5,INR 1000,55 Seats available
